# 06 correct lanes for reversible lanes with wrangler files

In [1]:
import os
import sys
import yaml
import pickle
import glob

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O and Parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_04')
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-02-22 22:37:06, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 22:37:06, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 22:37:06, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 22:37:06, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


## Read Version 05 Scenario

In [6]:
version_05_pickle_file_name = os.path.join(input_dir, 'working_scenario_05.pickle')
v_05_scenario = pickle.load(open(version_05_pickle_file_name, 'rb'))

In [7]:
v_05_scenario.road_net.links_df.shape

(2190435, 45)

In [8]:
v_05_scenario.road_net.links_df[
    v_05_scenario.road_net.links_df.model_link_id == 8016300].iloc[0].lanes

{'default': 3,
 'timeofday': [{'time': (21600, 36000), 'value': 2},
  {'time': (54000, 68400), 'value': 2}]}

## Create Scenario and Apply Project Cards

In [9]:
v_06_scenario = Scenario.create_scenario(
    base_scenario= v_05_scenario,
    card_directory = card_dir,
    glob_search = "*.wrangler",
    validate_project_cards=False)

2021-02-22 22:38:31, INFO: Creating Scenario
2021-02-22 22:38:31, INFO: Creating Scenario
2021-02-22 22:38:32, INFO: Adding project cards using glob search: *.wrangler
2021-02-22 22:38:32, INFO: Adding project cards using glob search: *.wrangler


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


In [10]:
v_06_scenario.apply_all_projects()

2021-02-22 22:38:32, INFO: Applying year 2015 golden gate reversible lanes
2021-02-22 22:38:32, INFO: Applying year 2015 golden gate reversible lanes
2021-02-22 22:38:32, INFO: Applying Project to Roadway Network: year 2015 golden gate reversible lanes
2021-02-22 22:38:32, INFO: Applying Project to Roadway Network: year 2015 golden gate reversible lanes


In [11]:
v_06_scenario.road_net.links_df[
    v_06_scenario.road_net.links_df.model_link_id == 8016300].iloc[0].lanes

{'default': 3, 'timeofday': [{'time': (54000, 68400), 'value': 1}]}

In [12]:
v_06_scenario.applied_projects

['Year 2015 Modify Facility Type to Service',
 'Year 2015 Modify Facility Type to Ramp shape incorrect',
 'Year 2015 Modify Facility Type to Ramp',
 'Year 2015 Modify Lanes to 2 Lane and Facility Type to Ramp shape incorrect',
 'Year 2015 Modify Lanes to 2 Lane and Facility Type to Ramp',
 'Year 2015 Modify Lanes to 1 Lane and Facility Type to Ramp',
 'Year 2015 Bus Only Segments',
 'Year 2015 Modify Lanes to 7 Lanes',
 'Year 2015 Modify Lanes to 6 Lanes',
 'Year 2015 Modify Lanes to 5 Lanes',
 'Year 2015 Modify Lanes to 4 Lanes',
 'Year 2015 Modify Lanes to 3 Lanes',
 'Year 2015 Modify Lanes to 2 Lanes',
 'Year 2015 Modify Lanes to 1 Lane',
 'Year 2015 Modify Lanes to 14 Lanes',
 'Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma',
 'Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma',
 'Year 2015 Modify HOV Lanes and USE on US 101 SB CA-85 to Oregon Expressway',
 'Year 2015 Modify HOV 

In [13]:
v_06_scenario.road_net.links_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2190435 entries, 0 to 2190434
Data columns (total 45 columns):
access                object
bike_access           object
drive_access          object
fromIntersectionId    object
lanes                 object
maxspeed              object
name                  object
oneWay                object
ref                   object
roadway               object
shstGeometryId        object
shstReferenceId       object
toIntersectionId      object
u                     object
v                     object
walk_access           object
wayId                 object
county                object
model_link_id         int64
A                     int64
B                     int64
rail_traveltime       object
rail_only             object
locationReferences    object
shape_id              object
geometry              geometry
ft_cal                object
ft                    int64
useclass              object
assignable            object
transit    

## Make Travel Model Networks

In [14]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_06_scenario.road_net, 
    parameters = parameters)

2021-02-22 22:44:33, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 22:44:33, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 22:44:33, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 22:44:33, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [15]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2190435, 45)
(688957, 12)
(1147816, 7)


In [16]:
model_net.links_df.model_link_id.nunique()

2190435

In [17]:
model_net = mtc.add_tap_and_tap_connector(
    roadway_network = model_net,
    parameters = parameters)

2021-02-22 22:44:34, INFO: Adding tap and tap connector to standard network
2021-02-22 22:44:34, INFO: Adding tap and tap connector to standard network
2021-02-22 22:44:34, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 22:44:34, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 22:44:34, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 22:44:34, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 22:44:39, INFO: Finished adding tap and tap connectors
2021-02-22 22:44:39, INFO: Finished adding tap and tap connectors


In [18]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2225677, 45)
(694957, 12)
(1165437, 7)


In [19]:
model_net.links_df.model_link_id.nunique()

2225677

In [20]:
model_net.nodes_df.model_node_id.nunique()

694957

In [21]:
model_net.shapes_df.id.nunique()

1165437

In [22]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2021-02-22 22:44:41, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-02-22 22:44:41, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-02-22 22:44:41, INFO: Creating managed lane network.
2021-02-22 22:44:41, INFO: Creating managed lane network.
2021-02-22 22:44:41, INFO: Creating network with duplicated managed lanes
2021-02-22 22:44:41, INFO: Creating network with duplicated managed lanes


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\wang

2021-02-22 23:00:53, INFO: Determining cntype
2021-02-22 23:00:53, INFO: Determining cntype
2021-02-22 23:00:53, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 23:00:53, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 23:00:53, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 23:00:53, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 23:00:53, INFO: Variable 'cntype' already in network. Returning without overwriting.
2021-02-22 23:00:53, INFO: Variable 'cntype' already in network. Returning without overwriting.
2021-02-22 23:00:53, INFO: Determining transit
2021-02-22 23:00:53, INFO: Determining transit
2021-02-22 23:00:53, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 23:00:53, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-02-22 23:00:53, INFO: Lasso base directory set as: Z:/Data/

In [23]:
model_net.links_df.shape

(2230079, 57)

In [24]:
model_net.links_df.model_link_id.nunique()

2230079

In [25]:
model_net.nodes_mtc_df.shape

(696464, 13)

In [26]:
model_net.nodes_df.model_node_id.nunique()

696464

## Write to Disk

## As shapefile

In [27]:
model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry", 
                             "drive_access",
                            'lanes_EA', 'lanes_AM',
       'lanes_MD', 'lanes_PM', 'lanes_EV'],
    node_output_variables = ["model_node_id", "geometry"],
    data_to_csv = False,
    data_to_dbf = True,
)

2021-02-22 23:08:27, INFO: Writing Network as Shapefile
2021-02-22 23:08:27, INFO: Writing Network as Shapefile
2021-02-22 23:08:27, INFO: Renaming DBF Node Variables
2021-02-22 23:08:27, INFO: Renaming DBF Node Variables
2021-02-22 23:08:27, INFO: Renaming variables so that they are DBF-safe
2021-02-22 23:08:27, INFO: Renaming variables so that they are DBF-safe
2021-02-22 23:08:39, INFO: Renaming DBF Link Variables
2021-02-22 23:08:39, INFO: Renaming DBF Link Variables
2021-02-22 23:08:39, INFO: Renaming variables so that they are DBF-safe
2021-02-22 23:08:39, INFO: Renaming variables so that they are DBF-safe
2021-02-22 23:08:42, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_04\nodes.shp
2021-02-22 23:08:42, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_04\nodes.shp
2021-02-22 23:09:25, INFO: Writing Link Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\pro

## As fixed width (for CUBE)

In [28]:
model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

2021-02-22 23:13:45, INFO: Starting fixed width conversion
2021-02-22 23:13:45, INFO: Starting fixed width conversion
2021-02-22 23:38:02, INFO: Writing out link database
2021-02-22 23:38:02, INFO: Writing out link database
2021-02-22 23:38:17, INFO: Writing out link header and width ----
2021-02-22 23:38:17, INFO: Writing out link header and width ----
2021-02-22 23:38:18, INFO: Starting fixed width conversion
2021-02-22 23:38:18, INFO: Starting fixed width conversion
2021-02-22 23:40:20, INFO: Writing out node database
2021-02-22 23:40:20, INFO: Writing out node database
2021-02-22 23:40:22, INFO: Writing out node header and width
2021-02-22 23:40:22, INFO: Writing out node header and width


In [29]:
%%capture
os.chdir(output_dir)
!make_cube.bat

NODEMERGE: 2179084

NODEMERGE: 2180084

NODEMERGE: 2181084

NODEMERGE: 2182084

NODEMERGE: 2183084

NODEMERGE: 2184084

NODEMERGE: 2185084

NODEMERGE: 2186084

NODEMERGE: 2187084

NODEMERGE: 2188084

NODEMERGE: 2189084

NODEMERGE: 2190084

NODEMERGE: 2191084

NODEMERGE: 2192084

NODEMERGE: 2500138

NODEMERGE: 2501138

NODEMERGE: 2502138

NODEMERGE: 2503138

NODEMERGE: 2504138

NODEMERGE: 2505138

NODEMERGE: 2506138

NODEMERGE: 2507138

NODEMERGE: 2508138

NODEMERGE: 2509138

NODEMERGE: 2510138

NODEMERGE: 2511138

NODEMERGE: 2512138

NODEMERGE: 2513138

NODEMERGE: 2514138



In [30]:
%%capture
!runtpp make_simple_roadway_network.s
!runtpp make_complete_marin_network.s
!runtpp make_simple_transit_network.s
!runtpp make_highway_ext_stn_connector_network.s

NODEMERGE: 2515138

NODEMERGE: 2516138

NODEMERGE: 2517138

NODEMERGE: 2518138

NODEMERGE: 2519138

NODEMERGE: 2520138

NODEMERGE: 2521138

NODEMERGE: 2522138

NODEMERGE: 2523138

NODEMERGE: 2524138

NODEMERGE: 2525138

NODEMERGE: 2526138

NODEMERGE: 2527138

NODEMERGE: 2528138

NODEMERGE: 2529138

NODEMERGE: 2530138

NODEMERGE: 2531138

NODEMERGE: 2532138

NODEMERGE: 2533138

NODEMERGE: 2534138

NODEMERGE: 2535138

NODEMERGE: 2536138

NODEMERGE: 2537138

NODEMERGE: 2538138

NODEMERGE: 2539138

NODEMERGE: 2540138

NODEMERGE: 2541138

NODEMERGE: 2542138

NODEMERGE: 2543138

NODEMERGE: 2544138

NODEMERGE: 2545138

NODEMERGE: 2546138

NODEMERGE: 2547138

NODEMERGE: 2548138

NODEMERGE: 2549138

NODEMERGE: 2550138

NODEMERGE: 2551138

NODEMERGE: 2552138

NODEMERGE: 2553138

NODEMERGE: 2554138

NODEMERGE: 2555138

NODEMERGE: 2556138

NODEMERGE: 2557138

NODEMERGE: 2558138

NODEMERGE: 2559138

NODEMERGE: 2560138

NODEMERGE: 2561138

NODEMERGE: 2562138

NODEMERGE: 2563138

NODEMERGE: 2564138



LINKMERGE: 410481-3094272     

LINKMERGE: 410842-3015719     

LINKMERGE: 411135-3073436     

LINKMERGE: 411509-3082123     

LINKMERGE: 411910-3056081     

LINKMERGE: 412290-3027567     

LINKMERGE: 412646-3067791     

LINKMERGE: 413096-3080671     

LINKMERGE: 413504-3028754     

LINKMERGE: 413914-3025928     

LINKMERGE: 414336-3024658     

LINKMERGE: 414783-3068998     

LINKMERGE: 415213-3083474     

LINKMERGE: 415650-3053620     

LINKMERGE: 416152-3031695     

LINKMERGE: 416642-3011775     

LINKMERGE: 417071-3092603     

LINKMERGE: 417511-3075539     

LINKMERGE: 418003-3008104     

LINKMERGE: 418465-3017105     

LINKMERGE: 418952-3039729     

LINKMERGE: 419418-3074899     

LINKMERGE: 419897-3080031     

LINKMERGE: 420397-3037745     

LINKMERGE: 420888-3017759     

LINKMERGE: 421456-3029090     

LINKMERGE: 422020-3082379     

LINKMERGE: 422547-3044803     

LINKMERGE: 423131-3075569     

LINKMERGE: 423599-3023065     

LINKMERGE: 424166-3093068     

LINKMERG

LINKMERGE: 2087555-2145746     

LINKMERGE: 2087868-2080610     

LINKMERGE: 2088172-2072849     

LINKMERGE: 2088481-2121308     

LINKMERGE: 2088783-2126031     

LINKMERGE: 2089083-2062767     

LINKMERGE: 2089385-2179868     

LINKMERGE: 2089703-2096529     

LINKMERGE: 2090014-2013946     

LINKMERGE: 2090305-2179952     

LINKMERGE: 2090613-2088755     

LINKMERGE: 2090919-2147467     

LINKMERGE: 2091215-228861     

LINKMERGE: 2091510-2154930     

LINKMERGE: 2091812-2124283     

LINKMERGE: 2092116-2069926     

LINKMERGE: 2092425-2060857     

LINKMERGE: 2092726-2132455     

LINKMERGE: 2093030-2092208     

LINKMERGE: 2093340-2123827     

LINKMERGE: 2093651-218233     

LINKMERGE: 2093950-2079611     

LINKMERGE: 2094250-2073414     

LINKMERGE: 2094560-2131351     

LINKMERGE: 2094865-2123652     

LINKMERGE: 2095174-2087323     

LINKMERGE: 2095484-2067479     

LINKMERGE: 2095794-2045168     

LINKMERGE: 2096095-2160416     

LINKMERGE: 2096410-2180653     

LINKMERGE: 2

## As transit line file (for CUBE)

In [33]:
standard_transit_net = StandardTransit.fromTransitNetwork(v_06_scenario.transit_net, parameters = parameters)
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

2021-02-11 18:28:53, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-11 18:28:53, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-11 18:28:53, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-11 18:28:53, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-11 18:28:53, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-02-11 18:28:53, INFO: Converting GTFS Standard Properties to MTC's Cube Standard


## As pickle

In [31]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_06.pickle')
pickle.dump(v_06_scenario, open(working_scenario_filename, 'wb'))

## As standard network

In [32]:
model_net.write(path = output_dir, filename = "v04")